[ver código](https://github.com/python-visualization/folium/blob/master/folium/plugins/heat_map_withtime.py)

### Objetivo

Criar uma mapa de calor do número de mortes causadas pela COVID-19 entre os dias 25-03-2020 e 11-04-2020 no estado de São Paulo.

### Origem dos Dados

Os dados foram extraídos dos boletins epidemiológicos que estão disponíveis na página [SITUAÇÃO EPIDEMIOLÓGICA DO ESTADO DE SÃO PAULO.](http://www.saude.sp.gov.br/cve-centro-de-vigilancia-epidemiologica-prof.-alexandre-vranjac/areas-de-vigilancia/doencas-de-transmissao-respiratoria/coronavirus-covid-19/situacao-epidemiologica) do site da prefeitura da cidade de São Paulo. Caso a página não esteja disponível, você poderá encontrar os pdfs dos boletins em questão clicando [aqui](https://github.com/SClovesgtx/SClovesgtx/tree/master/boletins).

Os dados parseados dos boletins podem ser obtidos em csv clicando [aqui](https://www.kaggle.com/clovesgtx/covid19-municpios-de-so-paulo).

In [1]:
import pandas as pd
import folium

In [2]:
df = pd.read_csv("data/covid-19-municipios-sp.csv")

In [3]:
df.Dia =  pd.to_datetime(df.Dia, format='%d-%m-%Y')

In [4]:
df.head()

,Dia,Boletim,Municipio,Confirmados,Obitos,Qt_Novos_Casos,Qt_Novas_Mortes,Latitude,Longitude
0,2020-03-25,30,BARUERI,1.0,0.0,NaN,NaN,-23.511218,-46.876461
1,2020-03-25,30,CAIEIRAS,1.0,0.0,NaN,NaN,-23.364462,-46.748477
2,2020-03-25,30,CAMPINAS,4.0,0.0,NaN,NaN,-22.905560,-47.060830
3,2020-03-25,30,CARAPICUIBA,2.0,0.0,NaN,NaN,-23.523467,-46.840681
4,2020-03-25,30,COTIA,6.0,0.0,NaN,NaN,-23.603889,-46.918889


## Mapa Base

In [5]:
def generateBaseMap(default_location=[-22.292690, -48.558171], default_zoom_start=7):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [6]:
base_map = generateBaseMap()
base_map

# Mapa de Calor

In [7]:
from folium.plugins import HeatMap

map_confirmados = generateBaseMap()
HeatMap(data=df[['Latitude', 'Longitude', 'Confirmados']].groupby(['Latitude', 'Longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(map_confirmados)
map_confirmados

# Adicionando Controlador de Tempo

In [20]:
df_day_list = []
for day in df.Dia.sort_values().unique():
    df_day_list.append(df.loc[df.Dia == day, ['Latitude', 'Longitude', 'Qt_Novas_Mortes']].groupby(['Latitude', 'Longitude']).sum().reset_index().values.tolist())

{heatmapOptions: {
                        radius: 13,
                        minOpacity: 0.05,
                        maxOpacity: 0.6,
                        scaleRadius: false,
                        useLocalExtrema: false,
                        defaultWeight: 1,
                        gradient: {0.4: 'blue', 0.65: 'lime', 1: 'red'}
                    }

In [23]:
from folium.plugins import HeatMapWithTime

mortes_confirmadas_map = generateBaseMap()
casos_covid_time = HeatMapWithTime(data=df_day_list, max_opacity=0.6,radius=13)
casos_covid_time.add_to(mortes_confirmadas_map)
mortes_confirmadas_map

In [24]:
casos_confirmados_map.save("covid-casos-confirmados-map-sp-tempo.html")

In [ ]:
py.plot(fig,filename='sampleplot.html)